### Prove of concept: we should be able to build an almost perfect prime model on modular features 
**modular feature**: is a number dividable by a given prime <br>
target whether int is/ isnt prime is almost a simple linear combination of features

In [ ]:
# imports
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

# sklearn imports
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegressionCV

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import ConfusionMatrixDisplay

# custom imports
from transform_fcts import (
    calculate_modular_features,
    binary_features,
    prime_distribution_features,
)


In [ ]:
# params
prime_lim = 500000

#### Build Core Dataset

In [ ]:
# read in prime numbers
primes = np.load(f'../../artifacts/primes/prime_{prime_lim}.npy')
primes[:100]

In [ ]:
# convert to natural numbers with binary target
natural_numbers = np.arange(0,prime_lim)
target = np.zeros(prime_lim, dtype=bool)
target[primes] = True

In [ ]:
data = pd.DataFrame(data={'n': natural_numbers[2:], 'y': target[2:]})
data.head()

### A: try out small data sets with modular features
--> if we can actually almost garantie a modular signal per prime in training, the models should show almost perfect performance

models: 
- prime cutoff 10000, and the lower 100 primes are used for features --> converges
- prime cutoff 100000, and the lower 100 primes are used for features --> converges
- prime cutoff 500000, and the lower 100 primes are used for features --> converges

In [ ]:
model_dict = {
    10000: {},
    100000: {},
    prime_lim: {},
} # prime_cutoff as key for models

n_modular_features = 100 # not all features
target_col = 'y'


for prime_cutoff in model_dict.keys():
    print(prime_cutoff,'\n')
    
    data_a = data[data['n']<prime_cutoff].copy()
    print(data_a.shape)
    
    # create modular features
    features = [data_a['n'].apply(lambda x: 1 if (x%prime==0 and x!=prime) else 0).values for prime in primes[:n_modular_features]]
    features = np.array(features).T
    feature_col = [f"mod_{str(prime)}" for prime in primes[:n_modular_features]]

    data_a = pd.concat([data_a, pd.DataFrame(features, columns=feature_col)], axis=1)

    print(data_a.head())

    # split in train & test    
    X, y = data_a[feature_col], data_a[target_col]
    print(target_col in feature_col)
    
    X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.33, random_state=42)
    
    # train logistic regression as start
    # lbfgs solver, l2 penalty
    clf = LogisticRegressionCV(cv=10, random_state=0, max_iter=500).fit(X_train, y_train)
    
    # store models and data
    model_dict[prime_cutoff]['data'] = data_a.copy()
    model_dict[prime_cutoff]['model'] = clf
    
    model_dict[prime_cutoff]['X_train'] = X_train.copy()
    model_dict[prime_cutoff]['X_test'] = X_test.copy()
    model_dict[prime_cutoff]['y_train'] = y_train.copy()
    model_dict[prime_cutoff]['y_test'] = y_test.copy()

    print('Training completed')


In [ ]:
# create predictions for evaluation of models

for prime_cutoff in model_dict.keys():
    curmod = model_dict[prime_cutoff]
    curmod['y_pred'] = curmod['model'].predict(curmod['X_test'])


#### Check overall performance of models

In [ ]:
for prime_cutoff in model_dict.keys():
    print(f'model with prime cutoff {prime_cutoff}')
    curmod = model_dict[prime_cutoff]
    print('confusion matrix \n', confusion_matrix(curmod['y_test'], curmod['y_pred']), '\n')

    print(classification_report(curmod['y_test'], curmod['y_pred']))


very few misclassification in every model

#### Confusion matrix depending on signal in modular features for models
**When do we have misclassifications?** <br>
-> all false positives (not prime, but predicted as prime) should have no signal in modular features (like no single modular feature = 1) <br>
-> all false negatives (prime, but not predicted as prime) cannot have any modular signal (as they are not prime) -> so how does this misclassification happen? <br>

<br> 
- ideally, the model would perfectly learn that no modular signal = prime -> that would eliminate all false negatives <br>
- introducing other features than just modular features would help to reduce the false positives

In [ ]:
for i, prime_cutoff in enumerate(model_dict.keys()):
    fig, ax = plt.subplots(1,2, figsize=(9, 3.5))

    curmod = model_dict[prime_cutoff]

    mod_features = curmod['model'].feature_names_in_

    # add new superposition of modular features to dataframe
    curmod['X_test']['any_mod'] = curmod['X_test'][mod_features].aggregate('sum',axis=1)>0

    # confusion matrix with any modular features
    cm_mod = confusion_matrix(curmod['y_test'][curmod['X_test']['any_mod']], curmod['y_pred'][curmod['X_test']['any_mod']], labels=curmod['model'].classes_)

    disp = ConfusionMatrixDisplay(confusion_matrix=cm_mod,
                              display_labels=curmod['model'].classes_)

    disp.plot(ax=ax[0])
    
    # confusion matrix without any modular features
    cm_nonmod = confusion_matrix(curmod['y_test'][curmod['X_test']['any_mod']==False], curmod['y_pred'][curmod['X_test']['any_mod']==False], labels=curmod['model'].classes_)


    disp = ConfusionMatrixDisplay(confusion_matrix=cm_nonmod,
                                  display_labels=curmod['model'].classes_)
    disp.plot(ax=ax[1])

    ax[0].set_title('Modular features = 1', size=10)
    ax[1].set_title('Modular features = 0', size=10)

    plt.suptitle(f"Model with prime cutoff {prime_cutoff}")
    plt.subplots_adjust(wspace=0.3, hspace=0.3)
    plt.show()


- first model has perfect classification
- second model classifies some primes with instead of modular features, but no modular feature = prime (and is true in test set)
- third model correctly classifies any number with any modular signal as "not prime" but misclassifies all which are prime although there is no modular signal
  --> we now have to find other features which might help with this false positive group

### B: lets try to move away from modular features
- modular features are trivial, because if we provide them all, the recognition of "prime / no prime" is a simple linear superposition <br>
- lets try to find other features and reduce modular features

**models:**
- prime cutoff 500000, and the lower 50 primes are used for features, no other features -> converges
- prime cutoff 500000, and the lower 50 primes are used for features, some other normalized features added
  -> converges, other features seem completely unimportant
- prime cutoff 500000, and the lower 10 primes are used for features and modular features for n+1 and n-1 are introduced, some other normalized features added
  -> converges, first model with kinda interesting mixture of true & false classification
  -> we reaching the area where the modular features of n are not enough to classify the prime, and the model has to rely on its other featuers as well

**how can we characterize the space of numbers which helps to find whether a number is a prime**

- distance to last prime: kinda gives info about the prime density (cannot be 0)
- distance between the previous 2 prime numbers? (thats prob just introduce noise)
- number of primes lower n: that feature will be repetitive for a lot of numbers
- we could have modular info about the neighbours of n: n+1, n-1 (eeh that might also just introduce noise)
- lets try that: modular info about the first ten primes, for n, n+1 and n-1

-> look afterwards at feature importance, like did anything have a minimal chance compared to modular info for n <br>
-> compare performance to modular model

In [ ]:
n_modular_features = 10 # not all features
target_col = 'y'

data_b = data[data['n']<prime_lim].copy()

In [ ]:
# create modular features

data_b = calculate_modular_features(data_b, n_distance=0, primes=primes, n_modular_features=n_modular_features)

# print(data_b.head())

In [ ]:
data_b = calculate_modular_features(data_b, n_distance=1)
data_b = calculate_modular_features(data_b, n_distance=-1)

In [ ]:
data_b.head()

In [ ]:
# cut off last row of data_b as n+1_mod_features are not defined for last row
data_b = data_b.iloc[:-1]

In [ ]:
# for further calculation
data_b['last_prime']=data_b['n'].apply(lambda x: primes[primes<x].max() if x!=2 else -1)
data_b['2nd_last_prime']=data_b['last_prime'].apply(lambda x: primes[primes<x].max() if x>2 else -1)

# non-modular features
data_b['primes_lower_n']=data_b['n'].apply(lambda x: len(primes[primes<x]) if x!=2 else 0)
data_b['distance_to_last_prime']=data_b.apply(lambda x: x['n']-x['last_prime'] if x['n']!=2 else -1, axis=1)
data_b['distance_between_last2primes']=data_b.apply(lambda x: x['last_prime']-x['2nd_last_prime'] if x['n']>3 else -1, axis=1)

#print(data_b[['n','last_prime', '2nd_last_prime','primes_lower_n','distance_to_last_prime', 'distance_between_last2primes']].head(20))

data_b.drop(columns=['last_prime','2nd_last_prime'], inplace=True)


In [ ]:
feature_col = data_b.columns.drop(target_col)
print(target_col in feature_col)
print(len(feature_col))

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data_b[feature_col] = scaler.fit_transform(data_b[feature_col])

In [ ]:
# split in train & test    
X, y = data_b[feature_col], data_b[target_col]

X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.33, random_state=42)

# train logistic regression as start
# lbfgs solver, l2 penalty
clf = LogisticRegressionCV(cv=10, random_state=0, max_iter=500).fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
print(classification_report(y_test, y_pred))

#### Feature importance inspection

Feature importance due to coefficients of regression

the coeff plot kinda indicates that the model has no idea what its doing tbh... <br> 
I expected the n_mod-features to outrank everything by far. surprised that the n+1-mod / n-1-mod features can keep up

In [ ]:
# coefs != feature importance...
feature_importance = pd.DataFrame(data={'features': clf.feature_names_in_, 'coeffs': np.abs(clf.coef_[0])})
coeff_importance = feature_importance.sort_values('coeffs', ascending=True)

Feature importance from mutual info (entropy of target expression in relation to a features different values)

In [ ]:
from sklearn.feature_selection import mutual_info_classif

# mutual info completely independent from model
mutual_info_out = mutual_info_classif(X_test, y_test, random_state=0)
feature_importance['mutual_info'] = mutual_info_out
mutual_info_importance = feature_importance.sort_values('mutual_info', ascending=True)

Permutation feature importance 

In [ ]:
from sklearn.inspection import permutation_importance

permut_out = permutation_importance(clf, X_test, y_test, n_repeats=10, random_state=0)
feature_importance['permutation_importance'] = permut_out['importances_mean']
permut_importance = feature_importance.sort_values('permutation_importance', ascending=True)

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(20,10))

coeff_importance.plot(x='features', y='coeffs', kind='barh', ax=ax[0], title='Regression coeffs')
mutual_info_importance.plot(x='features', y='mutual_info', kind='barh', ax=ax[1], title='Mutual info')
permut_importance.plot(x='features', y='permutation_importance', kind='barh', ax=ax[2], title='Permutation importance')

ax[1].yaxis.label.set_visible(False)
ax[2].yaxis.label.set_visible(False)

plt.subplots_adjust(wspace=0.7)

- Coeffs are very binary in their assessment of whats important and what is not -> all modular features are important, the rest is not
- Mutual Info vs Permutation actually give a nice glimpse into the model quality: the fact that we have a nice correlation here shows that the model is not completely hallucinating
- it kinda fails to give meaning to the distance_last_primes though -> has meaning for target, but fails to have impact on prediction in permutation


In [ ]:
fig, ax = plt.subplots(1,3, figsize=(12, 3))

feature_importance.plot(x='mutual_info', y='coeffs', kind='scatter', ax=ax[0], title='Mutual Info vs Coeffs')
feature_importance.plot(x='permutation_importance', y='coeffs', kind='scatter', ax=ax[1], title='Permutation vs Coeffs')
feature_importance.plot(x='mutual_info', y='permutation_importance', kind='scatter', ax=ax[2], title='Mutual Info vs Permutation')

plt.subplots_adjust(wspace=0.5)

### C: Only using modular features of n-neighbours (n+1, n-1, etc) and non-modular features

In [ ]:
n_modular_features = 20 # not all features
target_col = 'y'

data_c = data[data['n']<prime_lim].copy()

Modular features

In [ ]:
# create modular features

mod_feature_df = calculate_modular_features(data_c, 0, primes=primes, n_modular_features=n_modular_features)

In [ ]:
# modular features of neighbours of n

n_neighbours = 2

for n_dist in range(1, n_neighbours+1):
    data_c = calculate_modular_features(data_df=data_c, n_distance=n_dist, mod_df=mod_feature_df)
    data_c = calculate_modular_features(data_df=data_c, n_distance=-n_dist, mod_df=mod_feature_df)

data_c = data_c.iloc[:-n_neighbours]

Non-modular features

In [ ]:
# features of binary representation
data_c = binary_features(data_c)

In [ ]:
# prime distribution features
data_c = prime_distribution_features(data_c, primes)

In [ ]:
data_c.head()

In [ ]:
feature_col = data_c.columns.drop(target_col)
print(target_col in feature_col)
print(len(feature_col))

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data_c[feature_col] = scaler.fit_transform(data_c[feature_col])

In [ ]:
# split in train & test    
X, y = data_c[feature_col], data_c[target_col]

X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.33, random_state=42)

# train logistic regression as start
# lbfgs solver, l2 penalty
clf = LogisticRegressionCV(cv=10, random_state=0, max_iter=500).fit(X_train, y_train)

In [ ]:
from sklearn.metrics import f1_score

def binarize_prediction(y_test, y_pred):
    
    thresholds = np.arange(0.0, 1.0, 0.01)  # Test thresholds between 0 and 1
    
    best_f1 = 0
    # Iterate through different thresholds to find the best one
    for threshold in thresholds:
        # Convert probabilities to binary predictions
        y_pred_bin = (y_pred >= threshold).astype(int)
        
        # Calculate F1 score
        f1 = f1_score(y_test, y_pred_bin)
            
        # Check if this threshold gives a better F1 score
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
    
    return (y_pred >= best_threshold).astype(int)

In [ ]:
#y_pred_proba = clf.predict_proba(X_test)
#y_pred = binarize_prediction(y_test, y_pred)

y_pred = clf.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
print(classification_report(y_test, y_pred))

--> model decides to just perform bad on the positive class -> you see the effect of high class-imbalance here <br>

#### Feature importance inspection

In [ ]:
from sklearn.feature_selection import mutual_info_classif

# mutual info completely independent from model
mutual_info_out = mutual_info_classif(X_test, y_test, random_state=0)

feature_importance = pd.DataFrame(data={'features': clf.feature_names_in_, 'mutual_info': mutual_info_out})

mutual_info_importance = feature_importance.sort_values('mutual_info', ascending=True)

In [ ]:
from sklearn.inspection import permutation_importance

permut_out = permutation_importance(clf, X_test, y_test, n_repeats=10, random_state=0)
feature_importance['permutation_importance'] = permut_out['importances_mean']
permut_importance = feature_importance.sort_values('permutation_importance', ascending=True)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(13,15))

mutual_info_importance.plot(x='features', y='mutual_info', kind='barh', ax=ax[0], title='Mutual info')
permut_importance.plot(x='features', y='permutation_importance', kind='barh', ax=ax[1], title='Permutation importance')

ax[0].yaxis.label.set_visible(False)
ax[1].yaxis.label.set_visible(False)

plt.subplots_adjust(wspace=0.7)

--> not really happy with that. binary features have a high mutual info, but dont impact the model in the permutation performance